## Google Multimodal Embedding models

### Install and configuration

In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# %pip install --upgrade --quiet google-genai \
#                                 numpy \
#                                 scipy

Note: you may need to restart the kernel to use updated packages.


In [2]:
#Set environment variables
PROJECT_ID = "ai-hangsik" 
REGION = "us-central1"
# USE_VERTEX_AI = True 

In [3]:
!gcloud auth application-default login
!gcloud auth application-default set-quota-project {PROJECT_ID}

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=hG6KyIpH9O9z7E9lszSLZfURjdxVrZ&access_type=offline&code_challenge=W72OWdGCjsqqPxidm4qWHRx5vcr3ZP7UvDuSzLdb6-g&code_challenge_method=S256


Credentials saved to file: [/Users/hangsik/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "ai-hangsik" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.

Credentials saved to file: [/Users/hangsik/.config/gc

### Execution

In [5]:
import vertexai
from google.cloud import aiplatform
from vertexai.vision_models import Image, MultiModalEmbeddingModel
import time

In [7]:
# Login to Vertex AI

vertexai.init(project=PROJECT_ID, location=REGION)


# client = genai.Client(
#     vertexai=USE_VERTEX_AI,
#     project=PROJECT_ID,
#     location=REGION,)

In [6]:
# Calculate cosine similarity between two embedding arrays
def cosine_similarity(embed_1, embed_2):
  import numpy as np
  from scipy.spatial.distance import cosine

  embedding_1 = np.array(embed_1)
  embedding_2 = np.array(embed_2)

  cosine_similarity = 1 - cosine(embedding_1, embedding_2)
  print(f"Cosine similarity : {cosine_similarity:.4f}")


### Google Embedding models
* https://docs.cloud.google.com/vertex-ai/generative-ai/docs/model-reference/multimodal-embeddings-api

In [ ]:
# Generate embedding using given model, image path, and contextual text

def generate_multimodal_embedding(model, image_path, contextual_text):
  
	start_time = time.perf_counter_ns()

	model = MultiModalEmbeddingModel.from_pretrained(model)

	# https://docs.cloud.google.com/python/docs/reference/vertexai/latest/vertexai.preview.vision_models.MultiModalEmbeddingModel#vertexai_preview_vision_models_MultiModalEmbeddingModel_get_embeddings
	
	embeddings = model.get_embeddings(

		image=Image.load_from_file(image_path),
		contextual_text=contextual_text,
		dimension=1408, # dimension must be one of 128, 256, 512, 1408

	)

	end_time = time.perf_counter_ns()

	latency = (end_time - start_time)
	print(f"Latency (ns): {latency*1e-6:.2f} ms")

	return embeddings

#### multimodalembedding@001

In [ ]:
MODEL = "multimodalembedding@001"
IMAGE_PATH = "gs://cloud-samples-data/vertex-ai/llm/prompts/landmark1.png"
TEXT = "Colosseum"


embeddings = generate_multimodal_embedding(model = MODEL,
                                image_path = IMAGE_PATH,
                                contextual_text = TEXT) 

print(f"Generated embedding vector for image (length: {len(embeddings.image_embedding)}):")
print(f"Generated embedding vector for text  (length: {len(embeddings.text_embedding)}):")

# cosine_similarity(embed_1, embed_2)

Latency (ns): 4222.95 ms
Generated embedding vector for image (length: 1408):
Generated embedding vector for text  (length: 1408):


In [27]:
from vertexai.language_models import TextEmbeddingModel
    
def generate_text_embedding(model, contextual_text):

    
	start_time = time.perf_counter_ns() 

	model = TextEmbeddingModel.from_pretrained(model)

	# https://docs.cloud.google.com/vertex-ai/generative-ai/docs/model-reference/text-embeddings-api#parameter-list

	instances = [
		{
			"task_type": "RETRIEVAL_QUERY",
			"content": contextual_text,
			#"title": "content's title if needed"
		}
	]

	parameters = {
		"autoTruncate" : True,
		"outputDimensionality": 1024,  # dimension must be one of 128, 256, 512, 1024

	}

	embeddings = model.get_embeddings(

		instances=instances,
		parameters=parameters
		

	)

	end_time = time.perf_counter_ns()

	latency = (end_time - start_time)
	print(f"Latency (ns): {latency*1e-6:.2f} ms")

	return embeddings

In [28]:
MODEL = "text-multilingual-embedding-002"
TEXT = "고양이가 자전거를 타고 간다"


embeddings = generate_text_embedding(model = MODEL,
                                contextual_text = TEXT) 

print(f"Generated embedding vector for text (length: {len(embeddings.text_embedding)}):")



/Users/hangsik/Documents/GitHub/ai_workshop/.venv/lib/python3.12/site-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


TypeError: _TextEmbeddingModel.get_embeddings() got an unexpected keyword argument 'instances'

## End of Document